# Advanced Map Viewer with Weather & Coffee Shops

This notebook subscribes to MQTT and displays all walking persons on a live map.
It also:
- Shows 5 coffee shops around City Hall
- Reacts to weather changes: dark sky and dark basemap when it rains
- Publishes cafe locations for persons to discover
- Updates markers with their colors in real-time

**Usage:**
1. Run all cells to start the map viewer
2. Launch `weather_controller.ipynb` to start weather cycles
3. Launch one or more `person_walker_advanced.ipynb` notebooks with different names
4. Watch as weather changes affect the map appearance and persons move to cafes on rain!
5. When weather clears, persons resume their random walk

In [ ]:
import importlib
import json
import asyncio
from typing import Any, Dict, List

from IPython.display import display
import simulated_city.maplibre_live as maplibre_live
from simulated_city.config import load_config
from simulated_city.mqtt import MqttConnector, MqttPublisher

importlib.reload(maplibre_live)
LiveMapLibreMap = maplibre_live.LiveMapLibreMap

In [ ]:
# City Hall coordinates (Copenhagen)
CITY_HALL_LNGLAT = (12.5683, 55.6761)

# Coffee shops around City Hall (approximate locations)
COFFEE_SHOPS: List[Dict[str, Any]] = [
    {"id": "cafe-1", "name": "Café Nordic", "lng": 12.5699, "lat": 55.6763},
    {"id": "cafe-2", "name": "Kaffebar Central", "lng": 12.5669, "lat": 55.6758},
    {"id": "cafe-3", "name": "Espresso Corner", "lng": 12.5689, "lat": 55.6749},
    {"id": "cafe-4", "name": "Brew House", "lng": 12.5675, "lat": 55.6770},
    {"id": "cafe-5", "name": "Morning Joe", "lng": 12.5710, "lat": 55.6755},
]

print(f"✓ Defined {len(COFFEE_SHOPS)} coffee shops around City Hall")

In [ ]:
# Create and display the map
m = LiveMapLibreMap(center=CITY_HALL_LNGLAT, zoom=16.5, height="700px")
m.add_basemap("OpenStreetMap.Mapnik")
m.add_basemap("CartoDB.DarkMatter")
m.add_3d_buildings()
display(m)
print("✓ Map initialized")

In [ ]:
# Add initial basemap and sky (sunny state)
m.set_visibility("OpenStreetMap.Mapnik", True)
m.set_visibility("CartoDB.DarkMatter", False)
m.set_sky(sky_color="#88C6FC", horizon_color="#F0E4D4", fog_color="#FFFFFF")
print("✓ Basemap and sky initialized (sunny state)")

In [ ]:
# Add static coffee shop markers
def add_coffee_shops_to_map():
    """Add all coffee shops as black markers on the map."""
    for cafe in COFFEE_SHOPS:
        m.add_marker(
            cafe["lng"],
            cafe["lat"],
            name=cafe["id"],
            color="#1a1a1a",  # Black
            popup=cafe["name"]
        )
    print(f"✓ Added {len(COFFEE_SHOPS)} coffee shop markers")

add_coffee_shops_to_map()

In [ ]:
# Connect to MQTT broker
cfg = load_config()
connector = MqttConnector(cfg.mqtt, client_id_suffix="map-viewer-advanced")
connector.connect()
if not connector.wait_for_connection(timeout=10.0):
    raise RuntimeError("Failed to connect to MQTT broker")

publisher = MqttPublisher(connector)
print("✓ Connected to MQTT broker")

In [ ]:
# Track state
persons_seen = set()
current_weather = "sunny"

def on_person_location(client, userdata, message):
    """
    Callback when a person location message arrives.
    
    Expected message format:
    {
        "lng": float,
        "lat": float,
        "color": str,
        "name": str,
        "timestamp": float
    }
    """
    try:
        data = json.loads(message.payload.decode())
        name = data["name"]
        lng = data["lng"]
        lat = data["lat"]
        color = data["color"]
        
        # Track new persons
        if name not in persons_seen:
            persons_seen.add(name)
            print(f"  ✓ New person on map: {name} (color: {color})")
        
        # Update marker position with color
        marker_id = f"person-{name}"
        m.move_marker(marker_id, (lng, lat), color=color)
    
    except Exception as e:
        print(f"Error processing person location: {e}")

def on_weather_change(client, userdata, message):
    """
    Callback when weather changes.
    
    Expected message format:
    {
        "state": "sunny" | "rain",
        "timestamp": float,
        "cycle": int
    }
    """
    global current_weather
    try:
        data = json.loads(message.payload.decode())
        state = data["state"]
        current_weather = state
        
        if state == "rain":
            print(f"  🌧️  RAIN detected! Switching to dark mode...")
            # Set dark sky
            m.set_sky(sky_color="#010101")
            # Switch to dark basemap (hide light, add dark)
            try:
                m.set_visibility("OpenStreetMap.Mapnik", False)
                m.set_visibility("CartoDB.DarkMatter", True)
            except Exception:
                pass  # Layer might not exist yet

        elif state == "sunny":
            print(f"  ☀️  SUNNY! Switching back to light mode...")
            # Reset to light sky
            m.set_sky(sky_color="#88C6FC")
            # Switch back to light basemap
            try:
                m.set_visibility("OpenStreetMap.Mapnik", True)
                m.set_visibility("CartoDB.DarkMatter", False)
            except Exception:
                pass
    
    except Exception as e:
        print(f"Error processing weather: {e}")

# Set up MQTT callbacks
connector.client.on_message = on_person_location
connector.client.subscribe("persons/+/location", qos=0)
print("✓ Subscribed to persons/+/location")

# Use a second callback for weather
def on_message_dispatch(client, userdata, message):
    if message.topic.startswith("persons/"):
        on_person_location(client, userdata, message)
    elif message.topic == "weather/status":
        on_weather_change(client, userdata, message)

connector.client.on_message = on_message_dispatch
connector.client.subscribe("weather/status", qos=0)
print("✓ Subscribed to weather/status")
print("\nWaiting for updates...\n")

In [ ]:
# Publish cafe locations periodically so walkers can discover them
def publish_cafe_locations():
    """Publish all cafe locations to MQTT for persons to discover."""
    message = {
        "cafes": [
            {"id": c["id"], "name": c["name"], "lng": c["lng"], "lat": c["lat"]}
            for c in COFFEE_SHOPS
        ],
        "timestamp": __import__("time").time()
    }
    publisher.publish_json("cafes/locations", json.dumps(message), qos=0)

async def cafe_broadcaster():
    """Broadcast cafe locations every 5 seconds."""
    print("Starting cafe location broadcaster...")
    while True:
        publish_cafe_locations()
        await asyncio.sleep(5.0)

cafe_task = asyncio.create_task(cafe_broadcaster())
print("✓ Cafe broadcaster started")

In [ ]:
# Status updater
async def status_updater():
    """Periodically show how many persons are being tracked."""
    while True:
        await asyncio.sleep(15)
        if persons_seen:
            person_list = ", ".join(sorted(persons_seen))
            print(f"  📍 Tracking {len(persons_seen)} person(s): {person_list} (weather: {current_weather})")
        else:
            print(f"  ⏳ No persons detected yet (weather: {current_weather})")

status_task = asyncio.create_task(status_updater())
print("✓ Map viewer is running!")
print("Run the next cell to stop.")

In [ ]:
# Stop the viewer
cafe_task.cancel()
status_task.cancel()
connector.disconnect()
print("✓ Map viewer stopped.")